In [7]:
import os
import gpxpy
import json
import csv
from typing import List, Tuple, Dict

GPX_FOLDER = 'hikingbook_route'
TXT_FOLDER = 'txt_data'
OUTPUT_HTML = 'input_point_map.html'

# --- GPX 讀取 ---
def load_gpx_routes(folder: str) -> List[Tuple[str, List[Tuple[float, float]]]]:
    routes = []
    for filename in os.listdir(folder):
        if filename.endswith('.gpx'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r', encoding='utf-8') as gpx_file:
                gpx = gpxpy.parse(gpx_file)
                points = []
                for track in gpx.tracks:
                    for segment in track.segments:
                        for point in segment.points:
                            points.append((point.latitude, point.longitude))
                if points:
                    routes.append((filename, points))
    return routes

# --- TXT 讀取：每條路線的地標點 ---
def load_txt_points(folder: str) -> Dict[str, List[Dict]]:
    point_data = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            full_path = os.path.join(folder, filename)
            points = []
            with open(full_path, 'r', encoding='utf-8') as txt_file:
                reader = csv.DictReader(txt_file, delimiter='\t')
                for row in reader:
                    try:
                        lat = float(row['緯度'])
                        lon = float(row['經度'])
                        name = row['路標指示']
                        elev = row.get('海拔', '')
                        popup_text = f"{name} ({elev} m)" if elev else name
                        points.append({'lat': lat, 'lon': lon, 'popup': popup_text})
                    except Exception as e:
                        print(f"⚠️ 無法解析 {filename} 中的某行: {e}")
            point_data[filename.replace('.txt', '.gpx')] = points
    return point_data

# --- HTML 產出 ---
def generate_html(routes: List[Tuple[str, List[Tuple[float, float]]]],
                  txt_points: Dict[str, List[Dict]]):
    routes_json = json.dumps([
        {'name': name, 'points': coords, 'markers': txt_points.get(name, [])}
        for name, coords in routes
    ])

    html_template = f"""
<!DOCTYPE html>
<html lang="zh-Hant">
<head>
    <meta charset="UTF-8">
    <title>GPX 路線檢視器</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css" />
    <style>
        #map {{ height: 90vh; }}
        #checkboxes {{
            height: 10vh;
            overflow-x: auto;
            white-space: nowrap;
            background: #f9f9f9;
            padding: 5px;
        }}
        .route-option {{
            display: inline-block;
            margin-right: 10px;
        }}
    </style>
</head>
<body>
    <div id="map"></div>
    <div id="checkboxes"></div>

    <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
    <script>
        const map = L.map('map').setView([24.0, 121.0], 8);
        L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
            maxZoom: 18
        }}).addTo(map);

        const routes = {routes_json};
        const routeLayers = {{}};
        const markerGroups = {{}};

        routes.forEach(route => {{
            const latlngs = route.points.map(p => [p[0], p[1]]);
            const polyline = L.polyline(latlngs, {{ color: 'blue' }}).addTo(map);
            routeLayers[route.name] = polyline;

            const markers = [];
            route.markers.forEach(pt => {{
                const marker = L.marker([pt.lat, pt.lon]).bindPopup(pt.popup).addTo(map);
                markers.push(marker);
            }});
            markerGroups[route.name] = markers;

            map.fitBounds(polyline.getBounds());

            const cbContainer = document.getElementById('checkboxes');
            const wrapper = document.createElement('div');
            wrapper.className = 'route-option';

            const checkbox = document.createElement('input');
            checkbox.type = 'checkbox';
            checkbox.checked = true;
            checkbox.id = route.name;
            checkbox.onchange = () => {{
                if (checkbox.checked) {{
                    routeLayers[route.name].addTo(map);
                    markerGroups[route.name].forEach(m => m.addTo(map));
                }} else {{
                    map.removeLayer(routeLayers[route.name]);
                    markerGroups[route.name].forEach(m => map.removeLayer(m));
                }}
            }};

            const label = document.createElement('label');
            label.htmlFor = route.name;
            label.innerText = route.name;

            wrapper.appendChild(checkbox);
            wrapper.appendChild(label);
            cbContainer.appendChild(wrapper);
        }});
    </script>
</body>
</html>
"""
    with open(OUTPUT_HTML, 'w', encoding='utf-8') as f:
        f.write(html_template)
    print(f'✅ 已產生 {OUTPUT_HTML}')

if __name__ == "__main__":
    gpx_routes = load_gpx_routes(GPX_FOLDER)
    txt_points = load_txt_points(TXT_FOLDER)
    if not gpx_routes:
        print("⚠️ 找不到 GPX 路線")
    else:
        generate_html(gpx_routes, txt_points)


✅ 已產生 input_point_map.html
